In [ ]:
# طريقة تنزيل الداتا سيت لتعمل مع المشروع
# Kaggle API Key setup
from google.colab import files

# Upload the Kaggle API key file
uploaded = files.upload()

!pip install kaggle

# Move the Kaggle API key to the correct directory
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the MNIST dataset
!kaggle datasets download -d oddrationale/mnist-in-csv

# Unzip the dataset
!unzip mnist-in-csv.zip

In [31]:
from mnist import fashion_mnist 

In [32]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import time

In [33]:
x,y,x_test,y_test = fashion_mnist('FASHION_MNIST')
y_class = y 
t_class_test = y_test

[MNIST] Caching data at FASHION_MNIST
[MNIST] Found http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz in cache.
[MNIST] Found http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz in cache.
[MNIST] Found http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz in cache.
[MNIST] Found http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz in cache.


In [34]:
x = x.reshape(x.shape[0],784)
x_test = x_test.reshape(x_test.shape[0],784)
x = x / 255.0
x_test = x_test / 255.0
print(x.shape)

(60000, 784)


In [35]:
ohe = OneHotEncoder(sparse=False)

# Fit and transform the training and test labels to one-hot vectors
y = ohe.fit_transform(y.reshape(-1, 1))
y_test = ohe.transform(y_test.reshape(-1, 1))

C:\Users\LENOVO\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [42]:
#the code of the resilient algorithm 
class NN:
    #initialize new object of nueral network 
    def __init__(self, X,Y, L=1,N_l=256,epochs=10, lr=0.001):
        self.X = X
        self.Y = Y
        self.layer_sizes =np.array([self.X.shape[1]]+[N_l]*L+[self.Y.shape[1]]) 
        self.n_samples = self.X.shape[0]
        self.__init_weights()
        self.__init_pre_gradient()
        self.__init_learning_rate()
        self.__init_pre_change_rate()
        self.__init_gradient()
        self.__init_change_rate()
        self.train_loss = list()
        self.train_acc = list()
        self.train_time = list()
        self.tot_time = list()
        self.metrics = [self.train_loss,self.train_acc,self.train_time,self.tot_time]
        shuffle = np.random.permutation(self.n_samples)
        self.X , self.X_val = np.split(self.X,[int(0.8*self.n_samples)])
        self.Y , self.Y_val = np.split(self.Y,[int(0.8*self.n_samples)])
        self.n_samples = self.X.shape[0]
        self.val_loss = list()
        self.val_acc = list()

    def _positive_sigmoid(self,x):
        return 1 / (1 + np.exp(-x))


    def _negative_sigmoid(self,x):
        # Cache exp so you won't have to calculate it twice
        exp = np.exp(x)
        return exp / (exp + 1)
    
    
    def __sigmoid(self, x, derivative=False):
        if derivative:
            return self.sigmoid(x,False)*(1 - self.sigmoid(x,False))
        positive = x >= 0
        # Boolean array inversion is faster than another comparison
        negative = ~positive

        # empty contains juke hence will be faster to allocate than zeros
        result = np.empty_like(x)
        result[positive] = self._positive_sigmoid(x[positive])
        result[negative] = self._negative_sigmoid(x[negative])

        return result
    def __softmax(self,x):
        # Compute softmax along the rows of the input   
        exponent = np.exp(x)
        return exponent/exponent.sum(axis=1,keepdims=True)

    # cross entropy loss function
    def __loss(self,y_pred,y):
        # Compute the loss along the rows, averaging along the number of samples
        return ((-np.log(y_pred))*y).sum(axis=1).mean()

    # calculate the accurac
    def __accuracy(self,y_pred,y):  
        # Compute the accuracy along the rows, averaging along the number of samples
        return np.all(y_pred==y,axis=1).mean()

    def __sigmoid_prime(self,h):
        # Compute the derivative of sigmoid where h=sigmoid(x)
        return h*(1-h)


    def __init_gradient(self):
        self.gradient = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.gradient.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))

    def __init_pre_gradient(self):
        self.pre_gradient = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.pre_gradient.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))
            
    def __init_pre_change_rate(self):
        self.pre_change_rate = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.pre_change_rate.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))  
            
    def __init_change_rate(self):
        self.change_rate = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.change_rate.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))  
   
    def __init_learning_rate(self):
        self.learning_rate = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.learning_rate.append(np.full((self.layer_sizes[i],self.layer_sizes[i+1]),0.01))
    
    def __init_weights(self):
        self.weights = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.weights.append(np.random.uniform(-1,1,size=[self.layer_sizes[i],self.layer_sizes[i+1]]))
    
    def __init_layers(self,batch_size):
        # Initialize and allocate arrays for the hidden layer activations 
        self.__h = [np.empty((batch_size,layer)) for layer in self.layer_sizes]
        
    def __feed_forward(self,batch):
        # Perform a forward pass of `batch` samples (N_samples x N_features)
        h_l = batch
        self.__h[0] = h_l
        for i,weights in enumerate(self.weights):
            h_l = self.__sigmoid(h_l.dot(weights))
            self.__h[i+1]=h_l
        self.__out = self.__softmax(self.__h[-1])
    #the function that implements resilient algorithm 
    def resilient(self,prev_gradient,gradient,learning_rate,pre_change_rate) :         
        gradient_multiplication = gradient * prev_gradient
        learning_mult = np.where(gradient_multiplication < 0, 0.5, np.where(gradient_multiplication == 0, 1, 1.2))
        learning_rate = np.clip(learning_rate * learning_mult,1e-12,50)
        curr_change_rate = learning_rate * np.sign(gradient)
        np.copyto(curr_change_rate,pre_change_rate * -1, where=(gradient_multiplication < 0))
        prev_gradient = np.where(gradient_multiplication < 0, 0, gradient)
        
        return curr_change_rate
    
    # function for calculate backpropagation
    def __back_prop(self,batch_y):
        # Update the weights of the network through back-propagation
        delta_t = (self.__out - batch_y)*self.__sigmoid_prime(self.__h[-1])
        for i in range(1,len(self.weights)+1):
            self.gradient[-i] = (self.__h[-i-1].T.dot(delta_t))/self.batch_size
            self.change_rate[-i] = self.resilient(self.pre_gradient[-i],self.gradient[-i],self.learning_rate[-i],self.pre_change_rate[-i])
            delta_t = self.__sigmoid_prime(self.__h[-i-1])*(delta_t.dot(self.weights[-i].T))
        for i in range(1,len(self.weights)+1):
            self.weights[-i]-=self.change_rate[-i]
            
    def predict(self,X):
        # Generate a categorical, one-hot, prediction given an input X
        X = X
        self.__init_layers(X.shape[0])
        self.__feed_forward(X)
        return self.__to_categorical(self.__out)
    
    # make an evaluation for the test data 
    def evaluate(self,X,Y):
        # Evaluate the performance (accuracy) predicting on X with true labels Y
        prediction = self.predict(X)
        return self.__accuracy(prediction,Y)         
            
    def __to_categorical(self,x):  
        # Transform probabilities into categorical predictions row-wise, by simply taking the max probability
        categorical = np.zeros((x.shape[0],self.Y.shape[1]))
        categorical[np.arange(x.shape[0]),x.argmax(axis=1)] = 1
        return categorical

    # responsible for training the model
    def train(self,batch_size=16,epochs=25,lr=1.0):
        # Train the model with a given batch size, epochs, and learning rate. Store and print relevant metrics.
        self.lr = lr
        self.batch_size=batch_size
        for epoch in range(epochs):
            start = time.time()
            self.__init_layers(self.batch_size)
            shuffle = np.random.permutation(self.n_samples)
            train_loss = 0
            train_acc = 0
            X_batches = np.array_split(self.X[shuffle],self.n_samples/self.batch_size)
            Y_batches = np.array_split(self.Y[shuffle],self.n_samples/self.batch_size)
            for batch_x,batch_y in zip(X_batches,Y_batches):
                self.__feed_forward(batch_x)  
                train_loss += self.__loss(self.__out,batch_y)
                train_acc += self.__accuracy(self.__to_categorical(self.__out),batch_y)
                self.__back_prop(batch_y)
                
            train_loss = (train_loss/len(X_batches))
            train_acc = (train_acc/len(X_batches))
            self.train_loss.append(train_loss)
            self.train_acc.append(train_acc)
            
            train_time = round(time.time()-start,3)
            self.train_time.append(train_time)
            
            self.__init_layers(self.X_val.shape[0])
            self.__feed_forward(self.X_val)
            val_loss = self.__loss(self.__out,self.Y_val)
            val_acc = self.__accuracy(self.__to_categorical(self.__out),self.Y_val)
            self.val_loss.append(val_loss)
            self.val_acc.append(val_acc)
            
            tot_time = round(time.time()-start,3)
            self.tot_time.append(tot_time)
            
            print(f"Epoch {epoch+1}: loss = {train_loss.round(3)} | acc = {train_acc.round(3)} | val_loss = {val_loss.round(3)} |val_acc = {val_acc.round(3)}")
   

In [43]:
m = NN(x,y)

In [44]:
m.train(batch_size=256,epochs=25,lr=1.0)

Epoch 1: loss = 1.696 | acc = 0.695 | val_loss = 1.6 |val_acc = 0.798
Epoch 2: loss = 1.593 | acc = 0.817 | val_loss = 1.597 |val_acc = 0.821
Epoch 3: loss = 1.584 | acc = 0.833 | val_loss = 1.594 |val_acc = 0.832
Epoch 4: loss = 1.58 | acc = 0.841 | val_loss = 1.585 |val_acc = 0.842
Epoch 5: loss = 1.58 | acc = 0.846 | val_loss = 1.58 |val_acc = 0.843
Epoch 6: loss = 1.581 | acc = 0.847 | val_loss = 1.579 |val_acc = 0.848
Epoch 7: loss = 1.579 | acc = 0.85 | val_loss = 1.594 |val_acc = 0.842
Epoch 8: loss = 1.58 | acc = 0.85 | val_loss = 1.6 |val_acc = 0.848
Epoch 9: loss = 1.582 | acc = 0.852 | val_loss = 1.586 |val_acc = 0.849
Epoch 10: loss = 1.584 | acc = 0.853 | val_loss = 1.586 |val_acc = 0.852
Epoch 11: loss = 1.585 | acc = 0.855 | val_loss = 1.593 |val_acc = 0.847
Epoch 12: loss = 1.588 | acc = 0.852 | val_loss = 1.594 |val_acc = 0.853
Epoch 13: loss = 1.591 | acc = 0.853 | val_loss = 1.586 |val_acc = 0.852
Epoch 14: loss = 1.59 | acc = 0.854 | val_loss = 1.597 |val_acc = 0.84

In [46]:
m.evaluate(x_test,y_test)
# y.shape

0.8458

In [60]:
#the code of the backpropagation algorithm 
class NN:
    def __init__(self, X,Y, L=1,N_l=256,epochs=10, lr=0.001):
#     self.sizes = sizes
#     self.epochs = epochs 
#     self.lr = lr
        self.X = X
        self.Y = Y
        self.layer_sizes =np.array([self.X.shape[1]]+[N_l]*L+[self.Y.shape[1]]) 
        self.n_samples = self.X.shape[0]
        self.__init_weights()
        self.__init_pre_gradient()
        self.__init_learning_rate()
        self.__init_pre_change_rate()
        self.__init_gradient()
        self.__init_change_rate()
        self.train_loss = list()
        self.train_acc = list()
        self.train_time = list()
        self.tot_time = list()
        self.metrics = [self.train_loss,self.train_acc,self.train_time,self.tot_time]
        shuffle = np.random.permutation(self.n_samples)
        self.X , self.X_val = np.split(self.X,[int(0.8*self.n_samples)])
        self.Y , self.Y_val = np.split(self.Y,[int(0.8*self.n_samples)])
        self.n_samples = self.X.shape[0]
        self.val_loss = list()
        self.val_acc = list()
#     def __sigmoid(self,x):
#     # VCompute the sigmoid
#         return 1./(1.+np.exp(-x))

    def _positive_sigmoid(self,x):
        return 1 / (1 + np.exp(-x))


    def _negative_sigmoid(self,x):
        # Cache exp so you won't have to calculate it twice
        exp = np.exp(x)
        return exp / (exp + 1)
        
    def __sigmoid(self, x, derivative=False):
        if derivative:
            return self.sigmoid(x,False)*(1 - self.sigmoid(x,False))
        positive = x >= 0
        # Boolean array inversion is faster than another comparison
        negative = ~positive

        # empty contains juke hence will be faster to allocate than zeros
        result = np.empty_like(x)
        result[positive] = self._positive_sigmoid(x[positive])
        result[negative] = self._negative_sigmoid(x[negative])

        return result
    def __softmax(self,x):
        # Compute softmax along the rows of the input   
        exponent = np.exp(x)
        return exponent/exponent.sum(axis=1,keepdims=True)

    def __loss(self,y_pred,y):
        # Compute the loss along the rows, averaging along the number of samples
        return ((-np.log(y_pred))*y).sum(axis=1).mean()

    def __accuracy(self,y_pred,y):  
        # Compute the accuracy along the rows, averaging along the number of samples
        return np.all(y_pred==y,axis=1).mean()

    def __sigmoid_prime(self,h):
        # Compute the derivative of sigmoid where h=sigmoid(x)
        return h*(1-h)

#     def __init_prev_gradient(self):
#         self.prev_gradient = list()
#         for i in range(self.layer_sizes.shape[0]-1):
#             self.prev_gradient.append(np.zeros(size=[self.layer_sizes[i],self.layer_sizes[i+1]]))

    def __init_gradient(self):
        self.gradient = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.gradient.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))

    def __init_pre_gradient(self):
        self.pre_gradient = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.pre_gradient.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))
            
    def __init_pre_change_rate(self):
        self.pre_change_rate = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.pre_change_rate.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))  
            
    def __init_change_rate(self):
        self.change_rate = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.change_rate.append(np.zeros((self.layer_sizes[i],self.layer_sizes[i+1])))  
   
    def __init_learning_rate(self):
        self.learning_rate = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.learning_rate.append(np.full((self.layer_sizes[i],self.layer_sizes[i+1]),0.01))
    
    def __init_weights(self):
        self.weights = list()
        for i in range(self.layer_sizes.shape[0]-1):
            self.weights.append(np.random.uniform(-1,1,size=[self.layer_sizes[i],self.layer_sizes[i+1]]))
    
    def __init_layers(self,batch_size):
        # Initialize and allocate arrays for the hidden layer activations 
        self.__h = [np.empty((batch_size,layer)) for layer in self.layer_sizes]
        
    def __feed_forward(self,batch):
        # Perform a forward pass of `batch` samples (N_samples x N_features)
        h_l = batch
        self.__h[0] = h_l
        for i,weights in enumerate(self.weights):
            h_l = self.__sigmoid(h_l.dot(weights))
            self.__h[i+1]=h_l
        self.__out = self.__softmax(self.__h[-1])
    
        
#         return curr_change_rate
    def __back_prop(self,batch_y):
        # Update the weights of the network through back-propagation
        delta_t = (self.__out - batch_y)*self.__sigmoid_prime(self.__h[-1])
        for i in range(1,len(self.weights)+1):
            self.gradient[-i] = (self.__h[-i-1].T.dot(delta_t))/self.batch_size
#             self.change_rate[-i] = self.resilient(self.pre_gradient[-i],self.gradient[-i],self.learning_rate[-i],self.pre_change_rate[-i])
            delta_t = self.__sigmoid_prime(self.__h[-i-1])*(delta_t.dot(self.weights[-i].T))
        for i in range(1,len(self.weights)+1):
            self.weights[-i]-= self.gradient[-i]
            
    def predict(self,X):
        # Generate a categorical, one-hot, prediction given an input X
        X = X
        self.__init_layers(X.shape[0])
        self.__feed_forward(X)
        return self.__to_categorical(self.__out)
    
    def evaluate(self,X,Y):
        # Evaluate the performance (accuracy) predicting on X with true labels Y
        prediction = self.predict(X)
        return self.__accuracy(prediction,Y)         

            
    def __to_categorical(self,x):  
        # Transform probabilities into categorical predictions row-wise, by simply taking the max probability
        categorical = np.zeros((x.shape[0],self.Y.shape[1]))
        categorical[np.arange(x.shape[0]),x.argmax(axis=1)] = 1
        return categorical

    def train(self,batch_size=16,epochs=25,lr=1.0):
        # Train the model with a given batch size, epochs, and learning rate. Store and print relevant metrics.
        self.lr = lr
        self.batch_size=batch_size
        for epoch in range(epochs):
            start = time.time()
            self.__init_layers(self.batch_size)
            shuffle = np.random.permutation(self.n_samples)
            train_loss = 0
            train_acc = 0
            X_batches = np.array_split(self.X[shuffle],self.n_samples/self.batch_size)
            Y_batches = np.array_split(self.Y[shuffle],self.n_samples/self.batch_size)
            for batch_x,batch_y in zip(X_batches,Y_batches):
                self.__feed_forward(batch_x)  
                train_loss += self.__loss(self.__out,batch_y)
                train_acc += self.__accuracy(self.__to_categorical(self.__out),batch_y)
                self.__back_prop(batch_y)
                
            train_loss = (train_loss/len(X_batches))
            train_acc = (train_acc/len(X_batches))
            self.train_loss.append(train_loss)
            self.train_acc.append(train_acc)
            
            train_time = round(time.time()-start,3)
            self.train_time.append(train_time)
            
            self.__init_layers(self.X_val.shape[0])
            self.__feed_forward(self.X_val)
            val_loss = self.__loss(self.__out,self.Y_val)
            val_acc = self.__accuracy(self.__to_categorical(self.__out),self.Y_val)
            self.val_loss.append(val_loss)
            self.val_acc.append(val_acc)
            
            tot_time = round(time.time()-start,3)
            self.tot_time.append(tot_time)
            
            print(f"Epoch {epoch+1}: loss = {train_loss.round(3)} | acc = {train_acc.round(3)} | val_loss = {val_loss.round(3)} |val_acc = {val_acc.round(3)}")
   

In [61]:
m = NN(x,y)

In [62]:
m.train(batch_size=256,epochs=50,lr=1.0)

Epoch 1: loss = 1.791 | acc = 0.596 | val_loss = 1.688 |val_acc = 0.724
Epoch 2: loss = 1.669 | acc = 0.732 | val_loss = 1.655 |val_acc = 0.747
Epoch 3: loss = 1.646 | acc = 0.749 | val_loss = 1.639 |val_acc = 0.755
Epoch 4: loss = 1.632 | acc = 0.758 | val_loss = 1.629 |val_acc = 0.767
Epoch 5: loss = 1.624 | acc = 0.768 | val_loss = 1.622 |val_acc = 0.767
Epoch 6: loss = 1.617 | acc = 0.773 | val_loss = 1.617 |val_acc = 0.783
Epoch 7: loss = 1.612 | acc = 0.78 | val_loss = 1.612 |val_acc = 0.774
Epoch 8: loss = 1.607 | acc = 0.786 | val_loss = 1.608 |val_acc = 0.789
Epoch 9: loss = 1.604 | acc = 0.792 | val_loss = 1.606 |val_acc = 0.789
Epoch 10: loss = 1.601 | acc = 0.797 | val_loss = 1.603 |val_acc = 0.794
Epoch 11: loss = 1.598 | acc = 0.801 | val_loss = 1.601 |val_acc = 0.801
Epoch 12: loss = 1.595 | acc = 0.808 | val_loss = 1.598 |val_acc = 0.807
Epoch 13: loss = 1.593 | acc = 0.813 | val_loss = 1.596 |val_acc = 0.803
Epoch 14: loss = 1.591 | acc = 0.816 | val_loss = 1.596 |val_

In [63]:
m.evaluate(x_test,y_test)

0.8349

In [7]:
# Levenberg Marquardt algorithm

import numpy as np
import time
import matplotlib.pyplot as plt


def convert(imgf, labelf, outf, n):
    f = open(imgf, "rb")
    o = open(outf, "w")
    l = open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28 * 28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image) + "\n")
    f.close()
    o.close()
    l.close()


mnist_train_x = "train-images-idx3-ubyte"
mnist_train_y = "train-labels-idx1-ubyte"
mnist_test_x = "t10k-images-idx3-ubyte"
mnist_test_y = "t10k-labels-idx1-ubyte"

convert(mnist_train_x, mnist_train_y, "train.csv", 60000)
convert(mnist_test_x, mnist_test_y, "test.csv", 10000)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    return x * (1 - x)


def softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)


def one_hot_encode(labels):
    unique_labels = np.unique(labels)
    encoded_labels = np.zeros((len(labels), len(unique_labels)))
    for i, label in enumerate(labels):
        encoded_labels[i, label] = 1
    return encoded_labels


def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)


def accuracy_percentage(y_true, y_pred):
    correct_predictions = np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1)
    return np.mean(correct_predictions) * 100


def clip_gradients(gradients, threshold):
    clipped_gradients = []
    for gradient in gradients:
        clipped_gradients.append(np.clip(gradient, -threshold, threshold))
    return clipped_gradients


class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.biases_hidden = np.zeros((1, hidden_size))
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.biases_output = np.zeros((1, output_size))

    def forward_pass(self, X):
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.biases_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.output = softmax(np.dot(self.hidden_output, self.weights_hidden_output) + self.biases_output)
        return self.output

    def backward_pass(self, X, y, learning_rate, gradient_clip=None):
        output_error = y - self.output
        hidden_error = np.dot(output_error, self.weights_hidden_output.T) * sigmoid_derivative(self.hidden_output)

        # Update weights and biases
        self.weights_hidden_output += learning_rate * np.dot(self.hidden_output.T, output_error)
        self.biases_output += learning_rate * np.sum(output_error, axis=0, keepdims=True)
        self.weights_input_hidden += learning_rate * np.dot(X.T, hidden_error)
        self.biases_hidden += learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

        # Clip gradients if specified
        if gradient_clip is not None:
            gradients = [self.weights_input_hidden, self.biases_hidden,
                         self.weights_hidden_output, self.biases_output]
            clipped_gradients = clip_gradients(gradients, gradient_clip)
            self.weights_input_hidden, self.biases_hidden, \
                self.weights_hidden_output, self.biases_output = clipped_gradients

    def compute_jacobian(self, X):
        # Forward pass to compute activations
        self.forward_pass(X)

        # Compute derivatives for the sigmoid function
        sigmoid_deriv = sigmoid_derivative(self.hidden_output)

        # Compute derivatives for the softmax function
        softmax_deriv = np.apply_along_axis(self._softmax_derivative, 1, self.output)
        # Compute the Jacobian for the hidden layer
        jacobian_hidden = np.dot(self.weights_input_hidden, np.diag(sigmoid_deriv[0]))
        
        # Compute the Jacobian for the output layer
        jacobian_output = np.dot(self.weights_hidden_output, np.diag(softmax_deriv[0]))

        # Combine the two Jacobians
        jacobian = np.dot(jacobian_hidden, jacobian_output)
        return jacobian

    def _softmax_derivative(self, softmax_output):
        # Derivative of softmax is a bit complex since it depends on all other outputs
        s = softmax_output.reshape(-1, 1)
        return np.diagflat(s) - np.dot(s, s.T)
    def train(self, X, y, epochs, learning_rate, gradient_clip=None):
        for epoch in range(epochs):
            output = self.forward_pass(X)
            self.backward_pass(X, y, learning_rate, gradient_clip)
            error = mean_squared_error(y, output)
            acc = accuracy_percentage(y, output)
            print(f"Epoch {epoch + 1}/{epochs}, Error: {error}, Accuracy: {acc}%")

    def predict(self, X):
        return self.forward_pass(X)
    
# Load the training data
train_file = open("train.csv", "r")
train_list = train_file.readlines()
train_file.close()
train_data = [list(map(int, record.split(','))) for record in train_list]
train_data = np.array(train_data)

# Extract features and labels
X_train = train_data[:, 1:] / 255.0  # Normalize pixel values to between 0 and 1
y_train = one_hot_encode(train_data[:, 0])

# Initialize and train the neural network using Levenberg-Marquardt with gradient clipping
input_size = X_train.shape[1]
hidden_size = 256  # Increased hidden layer size
output_size = 10

neural_net = NeuralNetwork(input_size, hidden_size, output_size)
neural_net.train(X_train, y_train, epochs=50, learning_rate=0.001, gradient_clip=1.0)

# Load the testing data
test_file = open("test.csv", 'r')
test_list = test_file.readlines()
test_file.close()

test_data = [list(map(int, record.split(','))) for record in test_list]
test_data = np.array(test_data)

# Extract features and labels
X_test = test_data[:, 1:] / 255.0  # Normalize pixel values to between 0 and 1
y_test = one_hot_encode(test_data[:, 0])

# Make predictions on the test set
predictions = neural_net.predict(X_test)

# Evaluate the model on the test set
test_error = mean_squared_error(y_test, predictions)
test_acc = accuracy_percentage(y_test, predictions)
print(f'Test Error: {test_error}, Test Accuracy: {test_acc}')

Epoch 1/50, Error: 0.15234150484410414, Accuracy: 16.928333333333335%
Epoch 2/50, Error: 0.17816535077543438, Accuracy: 9.985%
Epoch 3/50, Error: 0.09078365324355787, Accuracy: 25.426666666666662%
Epoch 4/50, Error: 0.09977163674148777, Accuracy: 18.45%
Epoch 5/50, Error: 0.09941350536574904, Accuracy: 20.171666666666667%
Epoch 6/50, Error: 0.1027843102465564, Accuracy: 16.388333333333332%
Epoch 7/50, Error: 0.11110224690641374, Accuracy: 4.951666666666667%
Epoch 8/50, Error: 0.09205951503301177, Accuracy: 11.621666666666666%
Epoch 9/50, Error: 0.09972641143821134, Accuracy: 22.220000000000002%
Epoch 10/50, Error: 0.09594750042844677, Accuracy: 10.118333333333334%
Epoch 11/50, Error: 0.10293909682850327, Accuracy: 17.015%
Epoch 12/50, Error: 0.09435921030494773, Accuracy: 11.551666666666668%
Epoch 13/50, Error: 0.1076283857032868, Accuracy: 18.736666666666665%
Epoch 14/50, Error: 0.09162051161144597, Accuracy: 11.465%
Epoch 15/50, Error: 0.12395151825753206, Accuracy: 19.98%
Epoch 16/5